In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import json
import re
import geopandas as gpd
from shapely.geometry import Point
import numpy as np
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

In [2]:
with open(r"C:\Users\hunte\Documents\NNS-DDA10\python\data\census_api_key.json") as fi:
    credentials = json.load(fi)

In [3]:
api_key = credentials['api_key']

# bring in median income

In [4]:
endpoint = 'https://api.census.gov/data/timeseries/poverty/saipe?get=NAME,SAEMHI_PT&for=county:*&in=state:47&YEAR=2020&key={0}'.format(api_key)

response = requests.get(endpoint)

In [5]:
res = response.json()

In [6]:
income = pd.DataFrame(res)

In [7]:
headers = income.iloc[0]
income = income[1:]
income.columns = headers

In [8]:
income = income.rename(columns = {'NAME' : 'county_name',
                                  'SAEMHI_PT' : 'median_income'})

# bring in population

In [9]:
endpoint2 = 'https://api.census.gov/data/2020/dec/dhc?get=NAME,P1_001N&for=county:*&in=state:47&key={0}'.format(api_key)

response2 = requests.get(endpoint2)

In [10]:
tn_pop = pd.DataFrame(response2.json())

In [11]:
headers = tn_pop.iloc[0]
tn_pop = tn_pop[1:]
tn_pop.columns = headers

In [12]:
tn_pop = tn_pop.rename(columns = {'NAME' : 'county_name',
                                  'P1_001N' : 'total_pop'})

# Here we pull in population for each county for 65+. We will do multiple joins to get a combined table for all populations 65+

In [13]:
endpoint3 = 'https://api.census.gov/data/2020/dec/dhc?get=NAME,P12_020N&for=county:*&in=state:47&key={0}'.format(api_key)

response3 = requests.get(endpoint3)

In [14]:
male_65_66 = pd.DataFrame(response3.json())

In [15]:
headers = male_65_66.iloc[0]
male_65_66 = male_65_66[1:]
male_65_66.columns = headers

In [16]:
male_65_66 = male_65_66.rename(columns = {'NAME' : 'county_name',
                                          'P12_020N' : 'pop'})

In [17]:
male_65_66['age'] = '65-66 Male'

In [18]:
endpoint4 = 'https://api.census.gov/data/2020/dec/dhc?get=NAME,P12_021N&for=county:*&in=state:47&key={0}'.format(api_key)

response4 = requests.get(endpoint4)

In [19]:
male_67_69 = pd.DataFrame(response4.json())

In [20]:
headers = male_67_69.iloc[0]
male_67_69 = male_67_69[1:]
male_67_69.columns = headers

In [21]:
male_67_69 = male_67_69.rename(columns = {'NAME' : 'county_name',
                                          'P12_021N' : 'pop'})

In [22]:
male_67_69['age'] = '67-69 Male'

In [23]:
endpoint5 = 'https://api.census.gov/data/2020/dec/dhc?get=NAME,P12_022N&for=county:*&in=state:47&key={0}'.format(api_key)

response5 = requests.get(endpoint5)

In [24]:
male_70_74 = pd.DataFrame(response5.json())

In [25]:
headers = male_70_74.iloc[0]
male_70_74 = male_70_74[1:]
male_70_74.columns = headers

In [26]:
male_70_74 = male_70_74.rename(columns = {'NAME' : 'county_name',
                                          'P12_022N' : 'pop'})

In [27]:
male_70_74['age'] = '70-74 Male'

In [28]:
endpoint6 = 'https://api.census.gov/data/2020/dec/dhc?get=NAME,P12_023N&for=county:*&in=state:47&key={0}'.format(api_key)

response6 = requests.get(endpoint6)

In [29]:
male_75_79 = pd.DataFrame(response6.json())

In [30]:
headers = male_75_79.iloc[0]
male_75_79 = male_75_79[1:]
male_75_79.columns = headers

In [31]:
male_75_79 = male_75_79.rename(columns = {'NAME' : 'county_name',
                                          'P12_023N' : 'pop'})

In [32]:
male_75_79['age'] = '75-79 Male'

In [33]:
endpoint7 = 'https://api.census.gov/data/2020/dec/dhc?get=NAME,P12_024N&for=county:*&in=state:47&key={0}'.format(api_key)

response7 = requests.get(endpoint7)

In [34]:
male_80_84 = pd.DataFrame(response7.json())

In [35]:
headers = male_80_84.iloc[0]
male_80_84 = male_80_84[1:]
male_80_84.columns = headers

In [36]:
male_80_84 = male_80_84.rename(columns = {'NAME' : 'county_name',
                                          'P12_024N' : 'pop'})

In [37]:
male_80_84['age'] = '80-84 Male'

In [38]:
endpoint8 = 'https://api.census.gov/data/2020/dec/dhc?get=NAME,P12_025N&for=county:*&in=state:47&key={0}'.format(api_key)

response8 = requests.get(endpoint8)

In [39]:
male_85_plus = pd.DataFrame(response8.json())

In [40]:
headers = male_85_plus.iloc[0]
male_85_plus = male_85_plus[1:]
male_85_plus.columns = headers

In [41]:
male_85_plus = male_85_plus.rename(columns = {'NAME' : 'county_name',
                                              'P12_025N' : 'pop'})

In [42]:
male_85_plus['age'] = '85+ Male'

In [43]:
male_pop_65plus = pd.concat([male_65_66, male_67_69, male_70_74, male_75_79, male_80_84, male_85_plus])

In [44]:
male_pop_65plus_pop = male_pop_65plus[['county_name', 'pop']]

In [45]:
male_pop_65plus_pop['pop'] = male_pop_65plus_pop['pop'].astype('int64')

C:\Users\hunte\AppData\Local\Temp\ipykernel_2528\2690378458.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  male_pop_65plus_pop['pop'] = male_pop_65plus_pop['pop'].astype('int64')


In [46]:
male_pop_65plus_pop = male_pop_65plus_pop.groupby(['county_name'], as_index = False)['pop'].sum()

In [47]:
endpoint9 = 'https://api.census.gov/data/2020/dec/dhc?get=NAME,P12_044N&for=county:*&in=state:47&key={0}'.format(api_key)

response9 = requests.get(endpoint9)

In [48]:
female_65_66 = pd.DataFrame(response9.json())

In [49]:
headers = female_65_66.iloc[0]
female_65_66 = female_65_66[1:]
female_65_66.columns = headers

In [50]:
female_65_66 = female_65_66.rename(columns = {'NAME' : 'county_name',
                                              'P12_044N' : 'pop'})

In [51]:
female_65_66['age'] = '65-66 Female'

In [52]:
endpoint10 = 'https://api.census.gov/data/2020/dec/dhc?get=NAME,P12_045N&for=county:*&in=state:47&key={0}'.format(api_key)

response10 = requests.get(endpoint10)

In [53]:
female_67_69 = pd.DataFrame(response10.json())

In [54]:
headers = female_67_69.iloc[0]
female_67_69 = female_67_69[1:]
female_67_69.columns = headers

In [55]:
female_67_69 = female_67_69.rename(columns = {'NAME' : 'county_name',
                                              'P12_045N' : 'pop'})

In [56]:
female_67_69['age'] = '67-69 Female'

In [57]:
endpoint11 = 'https://api.census.gov/data/2020/dec/dhc?get=NAME,P12_046N&for=county:*&in=state:47&key={0}'.format(api_key)

response11 = requests.get(endpoint11)

In [58]:
female_70_74 = pd.DataFrame(response11.json())

In [59]:
headers = female_70_74.iloc[0]
female_70_74 = female_70_74[1:]
female_70_74.columns = headers

In [60]:
female_70_74 = female_70_74.rename(columns = {'NAME' : 'county_name',
                                              'P12_046N' : 'pop'})

In [61]:
female_70_74['age'] = '70-74 Female'

In [62]:
endpoint12 = 'https://api.census.gov/data/2020/dec/dhc?get=NAME,P12_047N&for=county:*&in=state:47&key={0}'.format(api_key)

response12 = requests.get(endpoint12)

In [63]:
female_75_79 = pd.DataFrame(response12.json())

In [64]:
headers = female_75_79.iloc[0]
female_75_79 = female_75_79[1:]
female_75_79.columns = headers

In [65]:
female_75_79 = female_75_79.rename(columns = {'NAME' : 'county_name',
                                              'P12_047N' : 'pop'})

In [66]:
female_75_79['age'] = '75-79 Female'

In [67]:
endpoint13 = 'https://api.census.gov/data/2020/dec/dhc?get=NAME,P12_048N&for=county:*&in=state:47&key={0}'.format(api_key)

response13 = requests.get(endpoint13)

In [68]:
female_80_84 = pd.DataFrame(response13.json())

In [69]:
headers = female_80_84.iloc[0]
female_80_84 = female_80_84[1:]
female_80_84.columns = headers

In [70]:
female_80_84 = female_80_84.rename(columns = {'NAME' : 'county_name',
                                              'P12_048N' : 'pop'})

In [71]:
female_80_84['age'] = '80-84 Female'

In [72]:
endpoint14 = 'https://api.census.gov/data/2020/dec/dhc?get=NAME,P12_048N&for=county:*&in=state:47&key={0}'.format(api_key)

response14 = requests.get(endpoint14)

In [73]:
female_85_plus = pd.DataFrame(response14.json())

In [74]:
headers = female_85_plus.iloc[0]
female_85_plus = female_85_plus[1:]
female_85_plus.columns = headers

In [75]:
female_85_plus = female_85_plus.rename(columns = {'NAME' : 'county_name',
                                                  'P12_048N' : 'pop'})

In [76]:
female_85_plus['age'] = '85+ Female'

In [77]:
female_pop_65plus = pd.concat([female_65_66, female_67_69, female_70_74,
                               female_75_79, female_80_84, female_85_plus])

In [78]:
female_pop_65plus_pop = female_pop_65plus[['county_name', 'pop']]

In [79]:
female_pop_65plus_pop['pop'] = female_pop_65plus_pop['pop'].astype('int64')

C:\Users\hunte\AppData\Local\Temp\ipykernel_2528\851512073.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  female_pop_65plus_pop['pop'] = female_pop_65plus_pop['pop'].astype('int64')


In [80]:
female_pop_65plus_pop = female_pop_65plus_pop.groupby('county_name', as_index = False)['pop'].sum()

# merge 65+ for both genders

In [81]:
tnpop_65plus = pd.merge(male_pop_65plus_pop, female_pop_65plus_pop,
        how = 'inner', on = 'county_name')

In [82]:
tnpop_65plus['pop_over_65'] = tnpop_65plus['pop_x'] + tnpop_65plus['pop_y']

In [83]:
tnpop_65plus = tnpop_65plus.rename(columns = {'pop_x' : 'male_pop',
                                              'pop_y' : 'female_pop'})

In [84]:
tnpop_65plus = tnpop_65plus.merge(tn_pop, how = 'inner',
                                   on = 'county_name')

In [85]:
tnpop_65plus = tnpop_65plus.drop(columns = {'state', 'county'})

In [86]:
tnpop_65plus['total_pop'] = tnpop_65plus['total_pop'].astype('int64')

In [87]:
tnpop_65plus['pct_over_65'] = round((tnpop_65plus['pop_over_65']/tnpop_65plus['total_pop'])*100,2)

In [88]:
tnpop_65plus['county_name'] = tnpop_65plus['county_name'].str[:-11]

In [89]:
tnpop_65plus = tnpop_65plus.merge(income, how = 'inner',
                                   on = 'county_name')

In [90]:
tnpop_65plus = tnpop_65plus.drop(columns = {'YEAR', 'state', 'county'})

In [91]:
tnpop_65plus['county_name'] = tnpop_65plus['county_name'].str[:-7]

In [92]:
tnpop_65plus.to_csv('data/visualization_data/tnpop_65plus.csv')

# We now have a dataframe showing tn population with percent over 65 along with median income